# AUTOGRAD: Auto Gradient
Pytorch menyebut package `autograd` pada pytorch merupakan bagian paling penting pada neural network. Mengingat tiap arsitektur neural networks pasti membutuhkan turunan/gradien. `autograd` bertugas menghitung turunan semua operasi yang dilakukan pada tensor

In [13]:
import torch

## Tensor
- `torch.Tensor` akan merekam riwayat gradien jika kita set atribut `.requires_grad` bernilai `True` dan bisa diakses lewat atribut `.grad`
- Riwayat gradien tensor berasal dari perhitungan otomatis semua gradien oleh fungsi `.backward()`
- Jika tidak menginginkan perekaman riwayat lagi maka bisa memanggil `.detach()`
- Atau melalui `with torch.no_grad():`, khususnya ini berguna saat finetune pretrained model karena ada yang perlu difreeze
- Selain `Tensor` komponen lain penyusun graf komputasi adalah `Function`, mirip-mirip tensorflow lah
- Untuk mengetahui `Function` apa yang membentuk suatu `Tensor` dapat dilihat pada atribut `.grad_fn`

### Rencana Implementasi
1. Membuat matriks 2x2 **X** dan rekam riwayat gradien
2. Operasikan **Y**=**X** + 2 dilanjutkan **Z** = 3 \* **Y** \* **Y**
3. Hitung mean dari semua elemen, anggap seperti agregasi loss \[cost function\] lalu hitung gradiennya

In [44]:
#### Step 1
x = torch.ones(2, 2, requires_grad=True)

#### Step 2
y = x + 2
z = 3 * y * y

#### Step 3
j = z.mean()
gradients = torch.tensor(1.) #harus dalam bentuk float
j.backward(gradients, retain_graph=True) #atau J.backward(), retain_graph penting supaya graf komputasi tidak dihapus
                                         #PyTorch membuat graf saat runtime bukan statis seperti tensorflow

print(x)
print(y.grad_fn, z.grad_fn, j.grad_fn)
print(x.grad)
x.grad.zero_() #hapus gradien pada x supaya tidak terakumulasi

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
<AddBackward object at 0x7f57d0e75048> <ThMulBackward object at 0x7f57d0e58ef0> <MeanBackward1 object at 0x7f57d0e589e8>
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


tensor([[0., 0.],
        [0., 0.]])

$j = \frac{1}{4} \sum_{i=1}^{4} z_i$ dan $z_i = 3(x+2)^2$ sehingga jika kita cari $\frac{dj}{dx_1}, \frac{dj}{dx_2}, \frac{dj}{dx_3}, \frac{dj}{dx_4}$ akan mendapatkan nilai 4.5  

`.zero_()` diperlukan karena secara default gradien akan terakumulasi. Maksud dari terakumulasi adalah jika kita memanggil `.backward()` sebanyak 5 kali maka yang ada pada `.grad` adalah lima kali dari hasil satu kali `.backward()`. Contohnya bisa dilihat pada [forum PyTorch ini](https://discuss.pytorch.org/t/why-do-we-need-to-set-the-gradients-manually-to-zero-in-pytorch/4903/12)  

Variabel `gradients` fungsinya mirip sebagai penimbang loss, tetapi karena `j` hanya terdiri satu elemen maka tidak perlu ditulis secara eksplisit. Kita lakukan dengan contoh lain

In [31]:
gradients2 = torch.tensor([[0.25, 0.5], [0.75, 1]])
print(z, gradients2)

z.backward(gradients2)
print(x.grad)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<ThMulBackward>) tensor([[0.2500, 0.5000],
        [0.7500, 1.0000]])
tensor([[ 4.5000,  9.0000],
        [13.5000, 18.0000]])


`gradients2` dapat dibayangkan sebagai turunan `j` terhadap `z` dengan notasi matematikanya $\frac{dj}{dz_1}, \frac{dj}{dz_2}, \frac{dj}{dz_3}, \frac{dj}{dz_4}$. Atau seperti yang saya jelaskan di atas dapat dilihat sebagai penimbang/importance dari tiap loss. Maka tiap loss didefinisikan sebagai $.25 * \frac{dz_1}{dx}, .5 * \frac{dz_2}{dx}, .75 * \frac{dz_3}{dx}, 1 * \frac{dz_4}{dx}$ Semoga jelas ya, ini berdasar pengetahuan saya setelah membaca [stackoverflow](https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments). Jika ada yang mau koreksi silahkan

Contoh lain penggunaan `gradients` ada di bawah ini. `.norm()` ini maksudnya adalah L2 norm, jika kurang dari 1000 nilai L2 norm-nya maka lakukan komputasi lagi

In [32]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)
print(x.grad)

tensor([-796.9212,  -53.3214,  621.4634], grad_fn=<MulBackward>)
tensor([ 102.4000, 1024.0000,    0.1024])


Seperti yang sudah disebutkan pada notebook sebelumnya tentang operasi inplace, `requires_grad` ini juga punya methodnya dengan argument defaultnya `True`. Lihat contoh di bawah

In [41]:
a = torch.randn(2, 2)
a.requires_grad_(False)
print(a.requires_grad)
a.requires_grad_()
print(a.requires_grad)

False
True


Terakhir, kita coba implementasi dari `with torch.no_grad():`

In [48]:
print((a ** 2).requires_grad) #semua output dari input yang atribut `requires_grad` bernilai True juga akan memiliki
                              #atribut `requires_grad` bernilai True
    
with torch.no_grad():
    print((a ** 2).requires_grad)

True
False
